**Method 1**
 * Step 1

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython import display

In [2]:
%load_ext Cython

In [3]:
%%cython -a
import numpy as np
def calculate_block_mean_image(image):
    
    # grab the image dimensions
    h = image.shape[0]
    w = image.shape[1]
    
    out = np.zeros((int(h/2),int(w/2),1), np.intc)
    
    # loop over the image, pixel by pixel
    for x in range(0, int(w/2)-1):
        for y in range(0, int(h/2)-1):
            new_y = np.mean([image[2*y, 2*x][0], image[2*y+1, 2*x][0], image[2*y, 2*x+1][0], image[2*y+1, 2*x+1][0]])
            out[y, x] = int(new_y)
                
    
    return out

In [4]:
%%cython -a
import numpy as np
import pandas as pd
def calculate_block_diff(bi, bg):
    out = np.abs(np.subtract(bg, bi))
    return out

In [5]:
%%cython -a
import numpy as np
def calculate_block_th(bd, th):
    return (bd > th)

In [6]:
%%cython -a
import numpy as np
def calculate_and(bb_old, bb_new):
    old_bool = np.array(bb_old, dtype=bool)
    new_bool = np.array(bb_new, dtype=bool)
    bb_sum = np.logical_and(old_bool, new_bool)
    bb_sum = bb_sum.astype(int)
    return bb_sum

In [7]:
%%cython -a
import numpy as np
import cv2
def draw_blobs(image, bb):
    h = bb.shape[0]
    w = bb.shape[1]
    
    for x in range(w):
        for y in range(h):
            if (bb[y, x] == 1):
                image = cv2.rectangle(image, (2*x-1, 2*y-1), (2*x+3, 2*y+3), (0, 0, 255), 2)
                
    return image

In [8]:
%%cython -a
import numpy as np
import cv2
def threshold_img(bb):
    h = bb.shape[0]
    w = bb.shape[1]
    
    out = np.zeros((int(h*2),int(w*2),3), np.uint8)
    
    for x in range(w):
        for y in range(h):
            if (bb[y, x] == 1):
                out[2*y, 2*x] = (255, 255, 255)
                out[2*y+1, 2*x] = (255, 255, 255)
                out[2*y, 2*x+1] = (255, 255, 255)
                out[2*y+1, 2*x+1] = (255, 255, 255)
                
    return out

In [9]:
%matplotlib inline

cap = cv2.VideoCapture('../video_train_set/train-4.avi')
ind = 0
while(cap.isOpened()):
    try:
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        yuv_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2YUV)
        #frame = cv2.putText(frame, str(ind), (50,250), cv2.FONT_HERSHEY_SIMPLEX, 10, (255, 0, 0), 5, cv2.LINE_AA)
        BI = calculate_block_mean_image(yuv_frame)
        th = 4
        if (ind == 0):
            BG = BI
            BD = calculate_block_diff(BI, BG)
            BB = calculate_block_th(BD, th)
            final_out = threshold_img(BB)
            last_BB = BB
        else:
            BD = calculate_block_diff(BI, BG)
            BB = calculate_block_th(BD, th)
            if (ind > 1):
                BB = calculate_and(last_BB, BB)
            final_out = threshold_img(BB)
            gray_out = cv2.cvtColor(final_out, cv2.COLOR_BGR2GRAY)
            threshold = cv2.threshold(gray_out, 1, 255,
                cv2.THRESH_BINARY_INV)[1] 
            threshold = cv2.bitwise_not(threshold)
            kernel = np.ones((3, 3), np.uint8)
            #threshold = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel, iterations=1)
            threshold = cv2.dilate(threshold, kernel, iterations=2)
            #frame = draw_blobs(frame, BB)
            contours, _ = cv2.findContours(threshold,
                cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            largestContour = np.array([[]])

            if len(contours) > 0:
                largestContour = max(contours, key=cv2.contourArea)
                #cv2.drawContours(frame, contours, -1, 255, 2)
                # get the unrotated bounding box that surrounds the contour
                x,y,w,h = cv2.boundingRect(largestContour)

                # draw the unrotated bounding box
                frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

            last_BB = BB
        frame = cv2.putText(frame, "Frame: " + str(ind), (25,500), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5, cv2.LINE_AA)
        plt.imshow(frame)

        display.clear_output(wait=True)
        display.display(plt.gcf())

    except KeyboardInterrupt:
        cap.release()
    ind = ind + 1

Error in callback <function flush_figures at 0x7fe778456af0> (for post_execute):


KeyboardInterrupt: 